In [ ]:
!pip install emoji --upgrade

     |████████████████████████████████| 174 kB 4.2 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=80aceaa9256fd79ee40bf0c3aa140471c7ac8730ea59cd98710bfc6e18b22132
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
import emoji
import json
import requests
import pandas as pd
import numpy as np
import regex
import re
import string
from collections import Counter
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import csv

In [ ]:
from google.colab import files
import io
uploaded = files.upload()
trump = pd.read_csv(io.BytesIO(uploaded["DT_Sample_1000_USA.csv"]))

Saving DT_Sample_1000_USA.csv to DT_Sample_1000_USA.csv


In [ ]:
trump = pd.read_csv("DT_Sample_1000_USA.csv")
trump.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,user_join_date,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,10/31/2020 18:37,1.320000e+18,I blame #Trump â€¼ï¸â€¼ï¸â€¼ï¸â€¼ï¸\n\n@re...,0,0,Twitter for iPhone,8.120000e+17,GO VOTE ðŸ‡ºðŸ‡¸â¤ï¸â€¼ï¸â€¼ï¸â€¼ï¸â¤ï¸...,Save_America4US,"We the people, must stand UNITED and take back...",12/22/2016 16:45,8493,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,52:54.5
1,11/2/2020 23:49,1.320000e+18,#Democratic #Pennsylvania #mayor endorses #Tru...,0,0,Twitter Web App,1.250000e+18,strange time,strangetime__,"Communism can HIDE in different forms, look at...",4/17/2020 20:27,128,USA,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,24:03.4
2,10/18/2020 17:14,1.320000e+18,TRUMP HOUSE 2020~\n\nIs this a haunted house o...,3,0,Twitter for iPhone,3.315967e+09,Andy Munoz ðŸŒŠ,andymunoz48,Producer: @rslshow of @kslsports Owner: @salta...,6/9/2015 21:06,780,"Salt Lake City, UT",40.759620,-111.886798,Salt Lake City,United States of America,North America,Utah,UT,33:01.0
3,10/29/2020 4:00,1.320000e+18,@LeahR77 Itâ€™s been going up from January up...,0,1,Twitter for iPhone,2.972392e+08,Vote Biden #USA,voteBiden46,#JesusIsLord #entrepreneur #RepublicansForBide...,5/12/2011 5:26,8765,USA,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,50:51.9
4,10/28/2020 12:39,1.320000e+18,@PhilipRucker @NicolleDWallace This is #WiseGu...,0,0,Twitter Web App,1.040000e+18,Bad Ratings - Show Cancelled,richardandjai,"Conservative, but not stupid...",9/6/2018 11:37,50,NY,43.156168,-75.844995,NaN,United States of America,North America,New York,NY,19:36.1


In [ ]:
from google.colab import files
import io
uploaded = files.upload()
biden = pd.read_csv(io.BytesIO(uploaded["JB_Sample_1000_USA.csv"]))

Saving JB_Sample_1000_USA.csv to JB_Sample_1000_USA.csv


In [ ]:
biden = pd.read_csv("JB_Sample_1000_USA.csv")
biden.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,user_join_date,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,10/30/2020 21:49,1.320000e+18,@ewarren #Biden may be walking into an ambush!...,0,0,Twitter Web App,8.742585e+08,Jonathan Greenberg,JournalistJG,Award-winning public interest investigative jo...,10/11/2012 20:16,3490,"New York, NY",40.712728,-74.006015,New York,United States of America,North America,New York,NY,10:13.6
1,10/17/2020 12:46,1.320000e+18,"Listen, I'm not going to say that #JoeBiden's ...",606,165,Twitter for Android,1.260000e+18,🌹Dr. Karin Baker-Thompkins for Congress🏳️‍🌈🌊🌹,ResitsTrump,#divorced mom to 2 #furbabys. Candidate #FL 28...,5/23/2020 17:40,13070,"Jacksonville Beach, FL",30.294686,-81.393140,NaN,United States of America,North America,Florida,FL,35:40.9
2,11/7/2020 16:43,1.330000e+18,Finally!!! 🇺🇸 \n\n#biden #bidenharis2020 #JoeB...,0,0,Twitter for iPhone,1.270000e+18,Sean R. Corder,SeanRCorder1,"When you look into my eyes, you’ll see an endl...",6/7/2020 0:05,8,"Milwaukee, WI",43.034993,-87.922497,Milwaukee,United States of America,North America,Wisconsin,WI,20:22.9
3,10/15/2020 12:09,1.320000e+18,Scary times in the USSA ..\n#morningjoe #CNN #...,0,0,Twitter Web App,6.229694e+08,Teresa † ن,BlackIrishI,Adore my children. Love animals. Former Air Fo...,6/30/2012 17:03,20999,Colo. (God's Country),38.725178,-105.607717,NaN,United States of America,North America,Colorado,CO,55:57.1
4,10/27/2020 18:07,1.320000e+18,"Watch ""Johnny Carson - Joe #Biden joke (Septem...",0,0,Plume for Android,2.826850e+08,Proud American Ok 🇺🇲 🐾🎃🕷🦇,Anne_R_u_Ok,NaN,4/15/2011 18:05,11747,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,36:13.5


In [ ]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text


call_emoji_free = lambda x: give_emoji_free_text(x)

trump['emoji_free'] = trump['tweet'].apply(give_emoji_free_text)
trump['url_free'] = trump['emoji_free'].apply(url_free_text)

trump.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,user_join_date,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at,emoji_free,url_free
0,10/31/2020 18:37,1.320000e+18,I blame #Trump â€¼ï¸â€¼ï¸â€¼ï¸â€¼ï¸\n\n@re...,0,0,Twitter for iPhone,8.120000e+17,GO VOTE ðŸ‡ºðŸ‡¸â¤ï¸â€¼ï¸â€¼ï¸â€¼ï¸â¤ï¸...,Save_America4US,"We the people, must stand UNITED and take back...",12/22/2016 16:45,8493,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,52:54.5,I blame #Trump â€¼ï¸â€¼ï¸â€¼ï¸â€¼ï¸ @realD...,I blame #Trump â€¼ï¸â€¼ï¸â€¼ï¸â€¼ï¸ @realD...
1,11/2/2020 23:49,1.320000e+18,#Democratic #Pennsylvania #mayor endorses #Tru...,0,0,Twitter Web App,1.250000e+18,strange time,strangetime__,"Communism can HIDE in different forms, look at...",4/17/2020 20:27,128,USA,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,24:03.4,#Democratic #Pennsylvania #mayor endorses #Tru...,#Democratic #Pennsylvania #mayor endorses #Tru...
2,10/18/2020 17:14,1.320000e+18,TRUMP HOUSE 2020~\n\nIs this a haunted house o...,3,0,Twitter for iPhone,3.315967e+09,Andy Munoz ðŸŒŠ,andymunoz48,Producer: @rslshow of @kslsports Owner: @salta...,6/9/2015 21:06,780,"Salt Lake City, UT",40.759620,-111.886798,Salt Lake City,United States of America,North America,Utah,UT,33:01.0,TRUMP HOUSE 2020~ Is this a haunted house or d...,TRUMP HOUSE 2020~ Is this a haunted house or d...
3,10/29/2020 4:00,1.320000e+18,@LeahR77 Itâ€™s been going up from January up...,0,1,Twitter for iPhone,2.972392e+08,Vote Biden #USA,voteBiden46,#JesusIsLord #entrepreneur #RepublicansForBide...,5/12/2011 5:26,8765,USA,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,50:51.9,@LeahR77 Itâ€™s been going up from January up ...,@LeahR77 Itâ€™s been going up from January up ...
4,10/28/2020 12:39,1.320000e+18,@PhilipRucker @NicolleDWallace This is #WiseGu...,0,0,Twitter Web App,1.040000e+18,Bad Ratings - Show Cancelled,richardandjai,"Conservative, but not stupid...",9/6/2018 11:37,50,NY,43.156168,-75.844995,NaN,United States of America,North America,New York,NY,19:36.1,@PhilipRucker @NicolleDWallace This is #WiseGu...,@PhilipRucker @NicolleDWallace This is #WiseGu...


In [ ]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text


call_emoji_free = lambda x: give_emoji_free_text(x)

biden['emoji_free'] = biden['tweet'].apply(give_emoji_free_text)
biden['url_free'] = biden['emoji_free'].apply(url_free_text)

biden.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,user_join_date,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at,emoji_free,url_free
0,10/30/2020 21:49,1.320000e+18,@ewarren #Biden may be walking into an ambush!...,0,0,Twitter Web App,8.742585e+08,Jonathan Greenberg,JournalistJG,Award-winning public interest investigative jo...,10/11/2012 20:16,3490,"New York, NY",40.712728,-74.006015,New York,United States of America,North America,New York,NY,10:13.6,@ewarren #Biden may be walking into an ambush!...,@ewarren #Biden may be walking into an ambush!...
1,10/17/2020 12:46,1.320000e+18,"Listen, I'm not going to say that #JoeBiden's ...",606,165,Twitter for Android,1.260000e+18,🌹Dr. Karin Baker-Thompkins for Congress🏳️‍🌈🌊🌹,ResitsTrump,#divorced mom to 2 #furbabys. Candidate #FL 28...,5/23/2020 17:40,13070,"Jacksonville Beach, FL",30.294686,-81.393140,NaN,United States of America,North America,Florida,FL,35:40.9,"Listen, I'm not going to say that #JoeBiden's ...","Listen, I'm not going to say that #JoeBiden's ..."
2,11/7/2020 16:43,1.330000e+18,Finally!!! 🇺🇸 \n\n#biden #bidenharis2020 #JoeB...,0,0,Twitter for iPhone,1.270000e+18,Sean R. Corder,SeanRCorder1,"When you look into my eyes, you’ll see an endl...",6/7/2020 0:05,8,"Milwaukee, WI",43.034993,-87.922497,Milwaukee,United States of America,North America,Wisconsin,WI,20:22.9,Finally!!! 🇺🇸 #biden #bidenharis2020 #JoeBiden...,Finally!!! 🇺🇸 #biden #bidenharis2020 #JoeBiden...
3,10/15/2020 12:09,1.320000e+18,Scary times in the USSA ..\n#morningjoe #CNN #...,0,0,Twitter Web App,6.229694e+08,Teresa † ن,BlackIrishI,Adore my children. Love animals. Former Air Fo...,6/30/2012 17:03,20999,Colo. (God's Country),38.725178,-105.607717,NaN,United States of America,North America,Colorado,CO,55:57.1,Scary times in the USSA .. #morningjoe #CNN #F...,Scary times in the USSA .. #morningjoe #CNN #F...
4,10/27/2020 18:07,1.320000e+18,"Watch ""Johnny Carson - Joe #Biden joke (Septem...",0,0,Plume for Android,2.826850e+08,Proud American Ok 🇺🇲 🐾🎃🕷🦇,Anne_R_u_Ok,NaN,4/15/2011 18:05,11747,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,36:13.5,"Watch ""Johnny Carson - Joe #Biden joke (Septem...","Watch ""Johnny Carson - Joe #Biden joke (Septem..."


In [ ]:
#Data Pre-processing, tokenization for topic modeling
# Load spacy, a pre-trained open source NLP model
# Make sure to restart the runtime after running installations and libraries tab
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)


# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']

# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

tokens = []

for doc in tokenizer.pipe(biden['url_free'], batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in STOP_WORDS:
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)

# Makes tokens column
biden['tokens'] = tokens

biden.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,user_join_date,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at,emoji_free,url_free,tokens
0,10/30/2020 21:49,1.320000e+18,@ewarren #Biden may be walking into an ambush!...,0,0,Twitter Web App,8.742585e+08,Jonathan Greenberg,JournalistJG,Award-winning public interest investigative jo...,10/11/2012 20:16,3490,"New York, NY",40.712728,-74.006015,New York,United States of America,North America,New York,NY,10:13.6,@ewarren #Biden may be walking into an ambush!...,@ewarren #Biden may be walking into an ambush!...,"[@ewarren, #biden, walking, ambush!, trump, ho..."
1,10/17/2020 12:46,1.320000e+18,"Listen, I'm not going to say that #JoeBiden's ...",606,165,Twitter for Android,1.260000e+18,🌹Dr. Karin Baker-Thompkins for Congress🏳️‍🌈🌊🌹,ResitsTrump,#divorced mom to 2 #furbabys. Candidate #FL 28...,5/23/2020 17:40,13070,"Jacksonville Beach, FL",30.294686,-81.393140,NaN,United States of America,North America,Florida,FL,35:40.9,"Listen, I'm not going to say that #JoeBiden's ...","Listen, I'm not going to say that #JoeBiden's ...","[listen,, going, #joebiden's, presidency, scan..."
2,11/7/2020 16:43,1.330000e+18,Finally!!! 🇺🇸 \n\n#biden #bidenharis2020 #JoeB...,0,0,Twitter for iPhone,1.270000e+18,Sean R. Corder,SeanRCorder1,"When you look into my eyes, you’ll see an endl...",6/7/2020 0:05,8,"Milwaukee, WI",43.034993,-87.922497,Milwaukee,United States of America,North America,Wisconsin,WI,20:22.9,Finally!!! 🇺🇸 #biden #bidenharis2020 #JoeBiden...,Finally!!! 🇺🇸 #biden #bidenharis2020 #JoeBiden...,"[finally!!!, 🇺🇸, #biden, #bidenharis2020, #joe..."
3,10/15/2020 12:09,1.320000e+18,Scary times in the USSA ..\n#morningjoe #CNN #...,0,0,Twitter Web App,6.229694e+08,Teresa † ن,BlackIrishI,Adore my children. Love animals. Former Air Fo...,6/30/2012 17:03,20999,Colo. (God's Country),38.725178,-105.607717,NaN,United States of America,North America,Colorado,CO,55:57.1,Scary times in the USSA .. #morningjoe #CNN #F...,Scary times in the USSA .. #morningjoe #CNN #F...,"[scary, times, ussa, .., #morningjoe, #cnn, #f..."
4,10/27/2020 18:07,1.320000e+18,"Watch ""Johnny Carson - Joe #Biden joke (Septem...",0,0,Plume for Android,2.826850e+08,Proud American Ok 🇺🇲 🐾🎃🕷🦇,Anne_R_u_Ok,NaN,4/15/2011 18:05,11747,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,36:13.5,"Watch ""Johnny Carson - Joe #Biden joke (Septem...","Watch ""Johnny Carson - Joe #Biden joke (Septem...","[watch, ""johnny, carson, joe, #biden, joke, (s..."


In [ ]:
#Lemmatization: Convert tokens to their root versions
biden['tokens_back_to_text'] = [' '.join(map(str, l)) for l in biden['tokens']]

def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text) 

    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_ != 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas 

In [ ]:
biden['lemmas'] = biden['tokens_back_to_text'].apply(get_lemmas)

In [ ]:
# Make lemmas a string again
biden['lemmas_back_to_text'] = [' '.join(map(str, l)) for l in biden['lemmas']]

# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # Removing url's
    pattern = r"http\S+"
    
    tokens = re.sub(pattern, "", text) # https://www.youtube.com/watch?v=O2onA4r5UaY
    tokens = re.sub('[^a-zA-Z 0-9]', '', text)
    tokens = re.sub('[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub('\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub('@*!*\$*', '', text) # Remove @ ! $
    tokens = re.sub('[Tt]rump|donaldtrump|@realdonaldtrump|donald|@realdonald|trump2020|[Bb]iden|joebiden|election2020|2020|bidenharris|harris|election|vote', '', text)
    tokens = tokens.strip(',') # TESTING THIS LINE
    tokens = tokens.strip('?') # TESTING THIS LINE
    tokens = tokens.strip('!') # TESTING THIS LINE
    tokens = tokens.strip("'") # TESTING THIS LINE
    tokens = tokens.strip(".") # TESTING THIS LINE

    tokens = tokens.lower().split() # Make text lowercase and split it
    
    return tokens

# Apply tokenizer
biden['lemma_tokens'] = biden['lemmas_back_to_text'].apply(tokenize)
biden.head(20)

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,user_join_date,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at,emoji_free,url_free,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,10/30/2020 21:49,1.320000e+18,@ewarren #Biden may be walking into an ambush!...,0,0,Twitter Web App,8.742585e+08,Jonathan Greenberg,JournalistJG,Award-winning public interest investigative jo...,10/11/2012 20:16,3490,"New York, NY",40.712728,-74.006015,New York,United States of America,North America,New York,NY,10:13.6,@ewarren #Biden may be walking into an ambush!...,@ewarren #Biden may be walking into an ambush!...,"[@ewarren, #biden, walking, ambush!, trump, ho...",@ewarren #biden walking ambush! trump hopes st...,"[@ewarren, biden, walk, ambush, trump, hope, s...",@ewarren biden walk ambush trump hope steal el...,"[@ewarren, walk, ambush, hope, steal, get, sup..."
1,10/17/2020 12:46,1.320000e+18,"Listen, I'm not going to say that #JoeBiden's ...",606,165,Twitter for Android,1.260000e+18,🌹Dr. Karin Baker-Thompkins for Congress🏳️‍🌈🌊🌹,ResitsTrump,#divorced mom to 2 #furbabys. Candidate #FL 28...,5/23/2020 17:40,13070,"Jacksonville Beach, FL",30.294686,-81.393140,NaN,United States of America,North America,Florida,FL,35:40.9,"Listen, I'm not going to say that #JoeBiden's ...","Listen, I'm not going to say that #JoeBiden's ...","[listen,, going, #joebiden's, presidency, scan...","listen, going #joebiden's presidency scandal. ...","[listen, go, joebiden, presidency, scandal, ye...",listen go joebiden presidency scandal yes ukra...,"[listen, go, presidency, scandal, yes, ukraine..."
2,11/7/2020 16:43,1.330000e+18,Finally!!! 🇺🇸 \n\n#biden #bidenharis2020 #JoeB...,0,0,Twitter for iPhone,1.270000e+18,Sean R. Corder,SeanRCorder1,"When you look into my eyes, you’ll see an endl...",6/7/2020 0:05,8,"Milwaukee, WI",43.034993,-87.922497,Milwaukee,United States of America,North America,Wisconsin,WI,20:22.9,Finally!!! 🇺🇸 #biden #bidenharis2020 #JoeBiden...,Finally!!! 🇺🇸 #biden #bidenharis2020 #JoeBiden...,"[finally!!!, 🇺🇸, #biden, #bidenharis2020, #joe...",finally!!! 🇺🇸 #biden #bidenharis2020 #joebiden...,"[finally, 🇺, 🇸, biden, bidenharis2020, joebide...",finally 🇺 🇸 biden bidenharis2020 joebiden byeb...,"[finally, 🇺, 🇸, haris, byebye, yourefire]"
3,10/15/2020 12:09,1.320000e+18,Scary times in the USSA ..\n#morningjoe #CNN #...,0,0,Twitter Web App,6.229694e+08,Teresa † ن,BlackIrishI,Adore my children. Love animals. Former Air Fo...,6/30/2012 17:03,20999,Colo. (God's Country),38.725178,-105.607717,NaN,United States of America,North America,Colorado,CO,55:57.1,Scary times in the USSA .. #morningjoe #CNN #F...,Scary times in the USSA .. #morningjoe #CNN #F...,"[scary, times, ussa, .., #morningjoe, #cnn, #f...",scary times ussa .. #morningjoe #cnn #foxnews ...,"[scary, time, ussa, morningjoe, cnn, foxnews, ...",scary time ussa morningjoe cnn foxnews msm big...,"[scary, time, ussa, morningjoe, cnn, foxnews, ..."
4,10/27/2020 18:07,1.320000e+18,"Watch ""Johnny Carson - Joe #Biden joke (Septem...",0,0,Plume for Android,2.826850e+08,Proud American Ok 🇺🇲 🐾🎃🕷🦇,Anne_R_u_Ok,NaN,4/15/2011 18:05,11747,United States,39.783730,-100.445882,NaN,United States,North America,NaN,NaN,36:13.5,"Watch ""Johnny Carson - Joe #Biden joke (Septem...","Watch ""Johnny Carson - Joe #Biden joke (Septem...","[watch, ""johnny, carson, joe, #biden, joke, (s...","watch ""johnny carson joe #biden joke (septembe...","[watch, johnny, carson, joe, biden, joke, sept...",watch johnny carson joe biden joke september 1...,"[watch, johnny, carson, joe, joke, september, ..."
5,10/23/2020 3:39,1.320000e+18,@AC360 @jaketapper #BidenWonTheDebate #BidenHa...,0,0,Twitter for iPhone,1.480003e+09,Todd Issacson,T889088,NaN,6/3/2013 15:40,120,"Washington, DC",38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,24:42.4,@AC360 @jaketapper #BidenWonTheDeb

In [ ]:
#Create a term-document matrix out of these lemmanized tokens
flat_list = []

for l in biden['lemma_tokens']:
  for item in l:
    flat_list.append(item)

vec = CountVectorizer(stop_words = flat_list)
counts = vec.fit_transform(biden['lemmas_back_to_text'])
counts = counts.toarray()

count_df = pd.DataFrame(counts, columns = vec.get_feature_names())
biden = pd.concat((biden, count_df), axis = 1)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['000', '03', '096', '1010wins', '10ish', '11thhour', '147', '15512927202', '163', '19', '200', '211', '220', '229', '22angiemarie22', '257', '263', '300', '30pm', '363', '400', '55', '60minutes', '6abc', '77', '82', '902', 'abbydphillip', 'abcnews', 'abcpolitic', 'abe_american', 'ac360', 'aceweedd', 'acosta', 'acyn', 'adamnagourney', 'all', 'alyssa_milano', 'amp', 'an', 'ananavarro', 'anandwrite', 'and', 'andrealynn789', 'andrewsolender', 'angiel806', 'antoniopolito1', 'aricformn', 'arifleischer', 'asb28504725', 'asharangappa', 'aspeninstitute', 'aux', 'avindman', 'bagaof4', 'ballots', 'bbc_arabic', 'bbcarabic', 'bbrian6', 'bcool333', 'beckyjohnsart', 'benshapiro', 'benwikler', 'betoorourke', 'bettemidler', 'bill19293640', 'billydees', 'bjapgar', 'bluemn', 'boston', 'bostondotcom', 'breakingn

In [ ]:
#Topic Model
from sklearn.decomposition import NMF
#Drop all other columns to only have term-document matrix
X = biden.iloc[ :,28:]
model = NMF(n_components = 4, init = "random", random_state = 0)
model.fit(X)

orders = np.argsort(model.components_, axis = 1)
orders

important_words = np.array(X.columns)[orders]
important_words

def top_words(X, model, component, num_words):
    orders = np.argsort(model.components_, axis = 1)
    important_words = np.array(X.columns)[orders]
    return important_words[component][-num_words:]

top_words(X, model, 3, 5)

array(['harris', 'realdonaldtrump', 'election2020', 'election', 'trump'],
      dtype=object)

In [ ]:
# Create a id2word dictionary
id2word = Dictionary(biden['lemma_tokens'])
print(len(id2word))
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in biden['lemma_tokens']]
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=1, id2word=id2word, workers=12, passes=5)
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
# Create Topics
topics = [' '.join(t[0:10]) for t in words]
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

4593
1472
------ Topic 0 ------
win president joe hunter @ kamala people day 🇺 go



In [ ]:
all_tweets = biden["tweet"].sum()
all_tweet_df = pd.DataFrame(data={"text": [all_tweets]})
f = open("alltweets.txt", "w")
f.write(all_tweet_df["text"][0])
f.close()

In [ ]:
# Term document matrix (towards datascience website)

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

corpus = ["alltweets.txt"]

vectorizer = TfidfVectorizer(stop_words = "english", ngram_range = (1,1), max_df = 1, min_df = 0)
X = vectorizer.fit_transform(corpus)

feature_names_out = vectorizer.get_feature_names()

dense = X.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns = feature_names_out)

df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,alltweets,txt
0,0.707107,0.707107


In [ ]:
#Install wordcloud

!pip install wordcloud

#Import packages

import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Prep TF-IDF Matrix

df_sum = df.sum(axis=0)

data = df_sum.transpose()

data.head()

alltweets    0.707107
txt          0.707107
dtype: float64